## Notebook: Calibration of the Heston Model  

This notebook focuses on pricing a **1-year call option** using the **Heston model** with the following parameters:  

- $v_0 = \sigma^2 = 0.05$  
- $\kappa = 0.5$  
- $\eta = 0.05$  
- $\theta = 0.2$  
- $\rho = -0.75$  
- $S_0 = 100$  
- $q = 0.01$ (dividend yield)  
- $r = 0.05$ (risk-free rate)  

### **Strike Prices**  
We consider three different strike prices:  
$$K = 90, 100, 110$$  

### **Pricing Methods**  
The option price is computed using the following methods:  
1. **Fast Fourier Transform (FFT)** with Simpson’s rule *(see Example Session 2)*.  
2. **Monte Carlo simulation** using the **Euler scheme**.  
3. **Monte Carlo simulation** using the **Milstein scheme**.  


In [2]:
# imports 
from tiny_pricing_utils import *
import numpy as np

In [3]:
# Market data
S0 = 100
q = 0.01
r = 0.05
sigma = 0.25
K = [90, 100, 110]
T = 1

In [4]:
# Hetson variables !!eta 
V0 = 0.05
kappa = 0.5 
eta = 0.05
theta = 0.2
rho = -0.75

In [5]:
# variables for FFT pricing (eta != eta_cm)
eta_cm = 0.25
N = 4096
alpha = 1.5
lambda_ = (2 * np.pi) / (N* eta_cm)
b = (N * lambda_) / 2
print(lambda_)
print(b)

0.006135923151542565
12.566370614359172


In [6]:
# Create an instance of the Heston model with the provided data
heston = HestonModel(S0=S0, r=r, q=q, V0=V0, kappa=kappa, eta=eta, theta=theta, rho=rho, T=T, 
                         alpha=alpha, N=N, eta_cm=eta_cm, b=b, strikes=K)

# Simpson's Rule coefficients for pricing
simpson_1 = 1 / 3  # First coefficient
simpson_weights = (3 + (-1) ** np.arange(2, N + 1)) / 3  # Alternating coefficients starting from index 2
simpson_weights = np.concatenate(([simpson_1], simpson_weights))  # Combine with the first coefficient

# Price options using Simpson's rule
option_prices_simpson = heston.price_options(rule="simpson", simpson_weights=simpson_weights)

# Print results for visual confirmation
print(f"Interpolated Option Prices using Simpson's Rule: {option_prices_simpson}")

Interpolated Option Prices using Simpson's Rule: [16.93747985 10.6163149   5.86707143]


In [7]:
#Monte Carlo pricing with Euler scheme
N = 252
M = 100000
np.random.seed(0)


In [8]:
def simulate_stock_paths_euler(S0, r, q, v0, kappa, eta, theta, rho, T, N, M):
    dt = T / N
    stock_paths = np.zeros((M, N + 1))
    v = np.zeros((M, N + 1))
    
    stock_paths[:, 0] = S0
    v[:, 0] = v0
    
    # Generate correlated random numbers
    epsilon1 = np.random.randn(M, N)
    epsilon2 = rho * epsilon1 + np.sqrt(1 - rho ** 2) * np.random.randn(M, N)
    
    for i in range(1, N + 1):
        v[:, i] = np.maximum(0, v[:, i-1] + kappa * (eta - v[:, i-1]) * dt 
                             + theta * np.sqrt(np.maximum(0, v[:, i-1]) * dt) * epsilon2[:, i-1])
        
        stock_paths[:, i] = stock_paths[:, i-1] * np.exp(
            (r - q - 0.5 * v[:, i-1]) * dt + np.sqrt(v[:, i-1] * dt) * epsilon1[:, i-1]
        )
    
    return stock_paths

In [11]:
def price_european_call_euler(S0, K, r, q, v0, kappa, eta, theta, rho, T, N, M):
    stock_paths = simulate_stock_paths_euler(S0, r, q, v0, kappa, eta, theta, rho, T, N, M)
    payoffs = np.maximum(stock_paths[:, -1] - K, 0)  # Use the last column
    option_price = np.exp(-r * T) * np.mean(payoffs)
    return option_price



In [12]:
# Compute prices for different strikes
for K in [90, 100, 110]:
    ec_price = price_european_call_euler(S0, K, r, q, V0, kappa, eta, theta, rho, T, N, M)
    print(f"European Call Option Price (K={K}): {ec_price:.2f}")


European Call Option Price (K=90): 16.97
European Call Option Price (K=100): 10.67
European Call Option Price (K=110): 5.88


In [ ]:
def simulate_stock_paths_milstein(S0, r, q, v0, kappa, eta, theta, rho, T, N, M):
    dt = T / N
    stock_paths = np.zeros((M, N + 1))
    v = np.zeros((M, N + 1))
    
    stock_paths[:, 0] = S0
    v[:, 0] = v0
    
    # Generate correlated random numbers
    epsilon1 = np.random.randn(M, N)
    epsilon2 = rho * epsilon1 + np.sqrt(1 - rho ** 2) * np.random.randn(M, N)
    
    for i in range(1, N + 1):
        sqrt_v = np.sqrt(np.maximum(0, v[:, i-1]))
        
        # Milstein correction term: 1/4 * theta^2 * (dW^2 - dt)
        milstein_correction = 0.25 * theta**2 * (epsilon2[:, i-1]**2 - 1) * dt
        
        # Update variance with Milstein correction
        v[:, i] = np.maximum(0, v[:, i-1] + kappa * (eta - v[:, i-1]) * dt 
                             + theta * sqrt_v * epsilon2[:, i-1] * np.sqrt(dt) 
                             + milstein_correction)
        
        # Update stock price using log-Euler
        stock_paths[:, i] = stock_paths[:, i-1] * np.exp(
            (r - q - 0.5 * v[:, i-1]) * dt + sqrt_v * epsilon1[:, i-1] * np.sqrt(dt)
        )
    
    return stock_paths

In [ ]:
def price_european_call_milstein(S0, K, r, q, v0, kappa, eta, theta, rho, T, N, M):
    stock_paths = simulate_stock_paths_milstein(S0, r, q, v0, kappa, eta, theta, rho, T, N, M)
    payoffs = np.maximum(stock_paths[:, -1] - K, 0)  # Use the last column
    option_price = np.exp(-r * T) * np.mean(payoffs)
    return option_price

In [ ]:
# Compute prices for different strikes
for K in [90, 100, 110]:
    mc_price_milstein = price_european_call_milstein(S0, K, r, q, V0, kappa, eta, theta, rho, T, N, M)
    print(f"Milstein European Call Option Price (K={K}): {mc_price_milstein:.2f}")